In [13]:
%matplotlib inline
import numpy as np
import matplotlib
import requests
import os
import ntpath
import pandas as pd
from time import sleep
matplotlib.rcParams['figure.figsize'] = (16.0,16.0)

In [14]:
URL = 'http://moca.usc.edu'
data = {'genome': 'hg19', 'format':'json'}
request_data = []

In [15]:
for file in os.listdir("/home/saket/Desktop/chip_scoring_beds/"):
    if file.endswith(".quest"):
        filename = "/home/saket/Desktop/chip_scoring_beds/{}".format(file)
        files = {'file': open(filename, 'rb')}
        fn = ntpath.basename(filename)
        req = requests.post(URL, files=files, data=data)
        try:
            resp = req.json()['job_id']
            request_data.append((fn, resp))
        except:
            print 'Failed: {}'.format(filename)
            print req.content
        sleep(.5)


Failed: /home/saket/Desktop/chip_scoring_beds/MCF-7_NEUROD1_4373S_Rep1vsRep2_RepAll_peaks.quest
<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.4.6 (Ubuntu)</center>
</body>
</html>

Failed: /home/saket/Desktop/chip_scoring_beds/K562_HDAC1_40967_Rep1-2_RepAll_peaks.quest
<html>
<head><title>504 Gateway Time-out</title></head>
<body bgcolor="white">
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.4.6 (Ubuntu)</center>
</body>
</html>



In [16]:
rurl = 'http://moca.usc.edu:5000/results/'
df = pd.DataFrame(columns = ('filename', 'link'))
for x, d in enumerate(request_data):
    df.loc[x] = [d[0], rurl+d[1]]

In [17]:
df
df.to_csv('encode_results.tsv', index=False, sep='\t')

In [18]:
for l in (df['link'].values):
    print l

http://moca.usc.edu:5000/results/b34620c6-b81e-4107-a972-b223a78f2171
http://moca.usc.edu:5000/results/a746042d-3aff-4fbf-a1a3-bb7162e87865
http://moca.usc.edu:5000/results/8a2a2b5c-e5a8-4c8f-93d3-95a1da28e729
http://moca.usc.edu:5000/results/a53b636d-e695-4e4e-bd94-731d25c9d851
http://moca.usc.edu:5000/results/3d1aa06d-1c92-4991-802a-13eaab808e05
http://moca.usc.edu:5000/results/09a413d9-daca-467b-ac3c-dc32d0865843
http://moca.usc.edu:5000/results/30f5ad5d-2c96-4d4b-a816-09f8662a96db
http://moca.usc.edu:5000/results/7007434c-748f-4804-a66b-6d916d52ff75
http://moca.usc.edu:5000/results/814e4eca-8155-4526-92f0-933bd0a7cada
http://moca.usc.edu:5000/results/06461a86-96dd-4bd0-9d5b-8118ba5a4544
http://moca.usc.edu:5000/results/e4f21f89-b4b0-4938-b7a0-6d110fcd0ce3
http://moca.usc.edu:5000/results/a866bb1b-b086-42fb-a74c-3df467abe153
http://moca.usc.edu:5000/results/55f39790-6273-4e3c-b1c3-5470559a1397
http://moca.usc.edu:5000/results/f57199cd-43b3-4561-b420-48c0688a7e21
http://moca.usc.edu: